In [ ]:
!pip install torch torchvision torchaudio \
    transformers diffusers gradio numpy pillow \
    cloudflared


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.2 MB/s eta 0:00:00
  

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00


In [2]:
!pip install better-profanity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.0 MB/s eta 0:00:00


In [3]:
import os
import random
import torch
import gradio as gr
import numpy as np
from PIL import Image, ImageEnhance
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from diffusers import StableDiffusionPipeline
from better_profanity import profanity
import requests
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)
comic_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

model_id = "prompthero/openjourney-v4"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16).to("cuda")
profanity.load_censor_words()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
GITHUB_WORDS_URL = "https://raw.githubusercontent.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/master/en"

In [5]:
def fetch_profanity_list():
    """Fetches an updated list of banned words from GitHub"""
    try:
        response = requests.get(GITHUB_WORDS_URL)
        response.raise_for_status()
        words = response.text.splitlines()
        return set(words)
    except requests.RequestException as e:
        print(f"Error fetching profanity list: {e}")
        return set()

EXPLICIT_WORDS = fetch_profanity_list()
profanity.load_censor_words()

art_styles = {
    "Classic Comic": (
        "Bold black ink outlines with vibrant primary colors (red, blue, yellow). Use halftone shading for depth and texture. "
        "Characters should have exaggerated action poses and dynamic compositions. "
        "Inspired by vintage Marvel and DC comics with a Jack Kirby aesthetic. "
        "Reference works: 'Fantastic Four' by Jack Kirby, 'Spider-Man' by Steve Ditko."
    ),
    "Anime": (
        "Sharp, clean lines with cel shading for a flat yet vibrant look. Use bright, saturated colors for characters and environments. "
        "Characters should have expressive faces and dynamic poses. Backgrounds should be detailed with a cinematic feel. "
        "Inspired by ‘Attack on Titan’ and ‘Demon Slayer.’ "
        "Reference works: 'Attack on Titan' by Hajime Isayama, 'Demon Slayer' by Koyoharu Gotouge."
    ),
    "Cartoon": (
        "Smooth, rounded edges with simplified shapes and bold outlines. Use a bright and cheerful color palette. "
        "Characters should have exaggerated expressions and humorous poses. Backgrounds should be colorful and stylized. "
        "Inspired by Disney, Pixar, and shows like ‘Gravity Falls.’ "
        "Reference works: 'Mickey Mouse' by Walt Disney, 'Gravity Falls' by Alex Hirsch."
    ),
    "Manga Style": (
        "High-contrast black-and-white with dramatic shading and speed lines for action scenes. "
        "Use screentones for texture and depth. Characters should have expressive faces and dynamic poses. "
        "Inspired by ‘One Piece,’ ‘Berserk,’ and ‘Naruto.’ "
        "Reference works: 'One Piece' by Eiichiro Oda, 'Berserk' by Kentaro Miura, 'Naruto' by Masashi Kishimoto."
    ),
    "Realism": (
        "Photorealistic shading with lifelike anatomy and highly detailed textures. Use natural colors and lighting. "
        "Characters and environments should look as realistic as possible, with attention to fine details. "
        "Inspired by AAA video games like ‘The Last of Us Part II.’ "
        "Reference works: 'The Last of Us Part II' by Naughty Dog, 'Red Dead Redemption 2' by Rockstar Games."
    ),
    "Studio Ghibli": (
        "Hand-painted aesthetic with lush, colorful backgrounds and soft textures. Use a pastel color palette with warm tones. "
        "Characters should be expressive yet organic, with a focus on emotions. Backgrounds should be detailed and whimsical. "
        "Inspired by ‘Spirited Away’ and ‘Howl’s Moving Castle.’ "
        "Reference works: 'Spirited Away' by Hayao Miyazaki, 'Howl’s Moving Castle' by Hayao Miyazaki."
    ),
    "Manwha": (
        "Vibrant colors with dynamic compositions and a focus on character emotions and interactions. "
        "Use detailed backgrounds with a mix of realism and stylized characters. Lighting should be dramatic and enhance the mood. "
        "Inspired by ‘Tower of God,’ ‘Solo Leveling,’ and ‘The God of High School.’ "
        "Reference works: 'Tower of God' by SIU, 'Solo Leveling' by Chugong, 'The God of High School' by Yongje Park."
    )
}

def contains_explicit_content(text):
    """Checks if text contains profane words from GitHub list and better_profanity."""
    lower_text = text.lower()
    words = lower_text.split()

    # Check using GitHub words list
    if any(word in EXPLICIT_WORDS for word in words):
        return True

    # Check using better_profanity library
    if profanity.contains_profanity(lower_text):
        return True

    return False

def generate_comic_story(topic, num_panels):
    """Generates a structured comic script with panel descriptions"""
    if contains_explicit_content(topic):
        return "Error: Inappropriate content detected. Please enter a suitable topic.", None

    instruction = (
        f"Create a {num_panels}-panel comic strip about {topic}. "
        "Each panel should be visually striking with clear actions, emotions, and interactions.\n\n"
        "Comic Strip Panels:\n"
    )

    response = comic_pipeline(
        instruction, max_new_tokens=400, temperature=0.7, repetition_penalty=1.1, do_sample=True
    )[0]['generated_text']

    comic_breakdown = response.replace(instruction, "").strip()
    comic_panels = [line.strip() for line in comic_breakdown.split("\n") if line.strip()][:num_panels]

    clean_panels = [profanity.censor(text) for text in comic_panels]  # Censor bad words in output

    return "\n".join(clean_panels)

def refine_prompt(description, style):
    """Refines panel descriptions into structured, detailed prompts for better image generation"""
    style_description = art_styles.get(style, "Comic style")
    return (
        f"{description}. The scene is visually dynamic with a strong focus on emotions and action. "
        f"Characters have expressive poses and facial expressions. The background is detailed, adding depth and realism. "
        f"Lighting and shading enhance the atmosphere. {style_description}."
    )

def enhance_image(image):
    """Enhances sharpness and contrast for clearer comic panels"""
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(2.0)

    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.3)

    return image

def generate_comic_images(edited_prompts, style, direction="LTR"):
    """Generates comic images from user-edited panel descriptions"""
    panels = edited_prompts.strip().split("\n")

    comic_images = [generate_comic_image(p, style) for p in panels]

    if not comic_images:
        return "No images were generated.", None

    # Reverse panels for RTL direction
    if direction == "RTL":
        comic_images = comic_images[::-1]

    # Organize panels into pages with a maximum of 9 panels per page (3x3 grid)
    pages = []
    for i in range(0, len(comic_images), 9):
        page_panels = comic_images[i:i+9]
        cols = 3
        rows = (len(page_panels) + cols - 1) // cols
        page = Image.new("RGB", (512 * cols, 512 * rows), "white")
        for j, img in enumerate(page_panels):
            if img:
                x_offset = (j % cols) * 512
                y_offset = (j // cols) * 512
                page.paste(img, (x_offset, y_offset))
        pages.append(page)

    return edited_prompts, pages

def generate_comic_image(description, style):
    """Generates a single comic panel using OpenJourney"""
    prompt = refine_prompt(description, style)
    negative_prompt = (
        "low quality, blurry, distorted faces, missing limbs, bad anatomy, extra fingers, watermark, text, unreadable details, "
        "washed out colors, lack of contrast, unnatural lighting"
    )

    try:
        image = pipe(
            prompt,
            negative_prompt=negative_prompt,
            width=512, height=512,
            num_inference_steps=40,
            guidance_scale=7.5
        ).images[0]

        return enhance_image(image)
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

with gr.Blocks() as interface:
    gr.Markdown("# COMICWALA")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a topic for the comic strip:", value="Enter a Prompt")
        num_panels_input = gr.Radio([3, 6, 9], label="Number of Comic Panels", value=3)
        direction_input = gr.Radio(["Left to Right (LTR)", "Right to Left (RTL)"], label="Comic Direction", value="Left to Right (LTR)")

    art_style_input = gr.Radio(list(art_styles.keys()), label="Choose an Art Style")

    generate_story_btn = gr.Button("Generate Story & Images")
    regenerate_images_btn = gr.Button("Regenerate Images (After Editing)")

    generated_story = gr.Textbox(label="Generated Story Prompts (Modify if needed)", lines=10)
    comic_strip_output = gr.Gallery(label="Generated Comic Pages")

    def generate_and_display_story(topic, num_panels, direction, style):
        """Generates the comic story and images"""
        story = generate_comic_story(topic, num_panels)
        if story.startswith("Error:"):
            return story, None

        direction = "LTR" if direction == "Left to Right (LTR)" else "RTL"
        _, comic_pages = generate_comic_images(story, style, direction)
        return story, comic_pages

    def regenerate_images(edited_story, direction, style):
        """Regenerates only the images from edited text"""
        direction = "LTR" if direction == "Left to Right (LTR)" else "RTL"
        return generate_comic_images(edited_story, style, direction)

    generate_story_btn.click(generate_and_display_story,
                            inputs=[topic_input, num_panels_input, direction_input, art_style_input],
                            outputs=[generated_story, comic_strip_output])

    regenerate_images_btn.click(regenerate_images,
                               inputs=[generated_story, direction_input, art_style_input],
                               outputs=[generated_story, comic_strip_output])

In [6]:
port = random.randint(10000, 60000)
interface.launch(server_name="0.0.0.0", server_port=port, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57471dc6fe7a8f5046.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
torch.cuda.empty_cache()